# https://www.yuque.com/ol1q37/gi94xp/vy59wr

# 已知有一些线上习题，https://pintia.cn/problem-sets/1111652100718116864/problems/type/7，
#采集所有题目（共95题，）
#1）存储所有要采集页面的完整html
#2）编写代码完成数据的抽取

In [22]:
from selenium import webdriver
import re
import time
driver=webdriver.Chrome()
url = "https://pintia.cn/problem-sets/1111652100718116864/problems/type/6"
driver.get(url)

# 创建SQL格式使数据工整

In [4]:
from pony.orm import *


db = Database()


class Topic(db.Entity):
    tab = PrimaryKey(str, column='tab', auto=True)
    Title = Optional(str, column='Title')
    grade = Optional(int, column='grade')
    Pass = Optional(int, column='Pass')
    Submit = Optional(int, column='Submit')
    Pass_Rate = Optional(str, column='Pass_Rate')
    html = Optional(str, column='html')
    Questions = Optional(str, column='Questions')
    tab_url = Optional(str, column='taburl')



#db.generate_mapping()

In [ ]:
db.bind(provider='sqlite', filename='E:/python/star.sqlite')

db.generate_mapping(create_tables=True)

set_sql_debug(True)

# 建立sql文件

In [5]:
db.bind(provider='sqlite', filename='E:/python/5.sqlite')
db.generate_mapping(create_tables=True)
set_sql_debug(True)

# 封装获取子网页的HTML和题目

In [30]:
def New_html(url):
    dr=webdriver.Chrome()
    dr.get(url)
    html = dr.page_source
    time.sleep(3)
    dr.close()
    return html

# 获取信息总函数，两个网页分别执行两次即可

In [31]:
def total(n):
    Questions="函数题"#题型
    #定义了一个正则表达式
    reobj = re.compile(r'<td class="answerIcon_1du7d"></td><td>([\d\D]*?)</td><td><a href="([\d\D]*?)"[\d\D]*?>([\d\D]*?)</a></td><td>([\d\D]*?)</td><td>([\d\D]*?)</td><td>([\d\D]*?)</td><td>([\d\D]*?)<')
    #迭代循环输出每一个匹配的数据
    for match in reobj.finditer(n):
        tab=match.group(1)#标号
        Title=match.group(3)#标题
        grade=match.group(4)#分数
        Pass=match.group(5)#通过数
        Submit=match.group(6)#提交数
        Pass_Rate=match.group(7)#通过率
        tab_url="https://pintia.cn"+match.group(2)#网址
        html=New_html(tab_url)#完整html
        s = Topic(tab = tab,Title = Title,grade=grade,Pass=Pass,Submit=Submit,Pass_Rate=Pass_Rate,html=html,Questions=Questions,tab_url=tab_url)
        db.commit()

In [ ]:
total(driver.page_source)
driver.find_element_by_xpath("//div[@id='sparkling-daydream']/div[3]/div[3]/div/ul/li[2]/a/div").click()
total(driver.page_source)

# 执行函数获取需要爬取的信息

## 采集https://pintia.cn/problem-sets/1111652100718116864/problems/type/6和https://pintia.cn/problem-sets/1111652100718116864/problems/type/7两个数据源（url1,url2)

# 第一步 获取到指定URL（url1,url2）网页源代码html,抽取出（标号	标题	分数	通过数	提交数	通过率 详情页链接,html，详情页的题目内容（content)），存入DB

# 第二步 遍历数据库中的每一条记录的详情页链接 ，利用selenium获取详情页网页源代码，并将其更新到DB中

# 第三步，遍历数据库抽取出题目的内容，更新到数据库相应的字段中